# Level 8: Introduction to SQLAlchemy (Core)

Welcome to SQLAlchemy! While writing raw SQL is powerful, it can become cumbersome and error-prone. SQLAlchemy is a popular Python library that provides a more 'Pythonic' way to interact with databases. It acts as an abstraction layer on top of raw SQL.

## 8.1 What is SQLAlchemy?

SQLAlchemy is composed of two distinct components:

1.  **SQLAlchemy Core (SQL Expression Language):** This is a toolkit that allows you to build SQL queries using Python objects and expressions. It provides a schema-centric view, where you think about tables, columns, and queries. It's a powerful way to write database-agnostic SQL in Python.

2.  **SQLAlchemy ORM (Object Relational Mapper):** This is built on top of the Core. It allows you to map your own Python classes to database tables and then interact with the database entirely through your objects, without writing any SQL. This is a more application-centric view.

**In this notebook, we focus on the Core.**

## 8.2 Installing SQLAlchemy

In [1]:
!pip install sqlalchemy


[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## 8.3 Connecting to SQLite

In SQLAlchemy, the central point of contact with the database is the **Engine**. You create it once per application.

In [2]:
from sqlalchemy import create_engine
import os

db_file = 'sqlalchemy_core.db'
if os.path.exists(db_file):
    os.remove(db_file)

# The connection string for SQLite is 'sqlite:///filename.db'
engine = create_engine(f'sqlite:///{db_file}')

print("SQLAlchemy engine created.")

SQLAlchemy engine created.


## 8.4 Metadata & Table Definition

In SQLAlchemy Core, you define your database schema using `MetaData` and `Table` objects.

In [3]:
from sqlalchemy import MetaData, Table, Column, Integer, String

# The MetaData object is a catalog of all our table definitions
metadata = MetaData()

# Define a 'users' table
users_table = Table('users',
    metadata,
    Column('id', Integer, primary_key=True),
    Column('name', String(50)), # String with a length
    Column('email', String)
)

print("Table object created:", users_table.name)

Table object created: users


### Creating the Table in the Database
The `Table` object is just a Python definition. To actually create it in the database, we use `metadata.create_all()`.

In [4]:
metadata.create_all(engine)
print("Table created in the database.")

Table created in the database.


## 8.5 Executing SQL Expressions

Now we can use our `Table` object to build queries.

In [5]:
from sqlalchemy import insert, select

# Create an insert statement
stmt = insert(users_table).values(name='Alice', email='alice@example.com')
print("Generated SQL:", stmt)

Generated SQL: INSERT INTO users (name, email) VALUES (:name, :email)


In [6]:
# To execute, we need a connection from the engine
with engine.connect() as conn:
    result = conn.execute(stmt)
    conn.commit() # Commit the transaction
    print(f"Insert successful. Primary key of new row: {result.inserted_primary_key}")

Insert successful. Primary key of new row: (1,)


### Building a SELECT statement

In [7]:
# Build a select statement to get all users
select_stmt = select(users_table)
print("Generated SQL:", select_stmt)

Generated SQL: SELECT users.id, users.name, users.email 
FROM users


In [8]:
with engine.connect() as conn:
    result = conn.execute(select_stmt)
    for row in result:
        print(row)

(1, 'Alice', 'alice@example.com')


### Filtering
You can add a `where` clause to your select statement.

In [9]:
select_where_stmt = select(users_table).where(users_table.c.name == 'Alice')
print("Generated SQL:", select_where_stmt)

with engine.connect() as conn:
    result = conn.execute(select_where_stmt)
    print("\nResult:", result.fetchone())

Generated SQL: SELECT users.id, users.name, users.email 
FROM users 
WHERE users.name = :name_1

Result: (1, 'Alice', 'alice@example.com')
